<a href="https://colab.research.google.com/github/zuzka05/stat_learn/blob/main/QuantTradingAccelerator_Part7_CrossValidation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quant Trading Accelerator 🚀

Learn from 0, extremely fast => JIT Learning => Build, Test, Learn, Iterate 🚀

Feedback is important!

## Part 7: Cross Validation

array => vector => time series => matrices => multi-variate time series => model => regression => classification => cross validation

## Cross-Validation Methods for Time-Series Data

1. Time Series Split
2. Expanding Window
3. Rolling Window



Goals:

1. Show how to get more reliable and robust estimate of how well a ML model will perform on new, unseen data
2. Compare model performance using cross validation using EW, RW and time-series split

In [1]:
# numerical computing library for fast calculation
import numpy as np
# data analysis library
import pandas as pd
# data visualization library
import seaborn as sns

# machine learning library
import torch
import torch.nn as nn
import torch.optim as optim

# needed for adding a fixed seed for our RNGs
import random
import os

### Time Split into Train/Test

Problem: can be sensitive to changes in test/train ratio


In [2]:
# time:  t0 ---- t1 ---- t2 ---- t3 ---- t4 ---- t5 ---- t6 ---- t7
# train: [===============================]
# test:                                  [=========================]

### Expanding Window

In [ ]:
#We increase the window size for our train set

In [3]:
# Time Series Data: [==============================]
#                    t1  t2  t3  t4  t5  t6  t7  t8


# Fold 1:
# Train:  [#####]
# Test:          [----]
#          t1 t2 t3 t4

# Fold 2:
# Train:  [###########]
# Test:                [----]
#          t1 t2 t3 t4 t5 t6

# Fold 3:
# Train:  [##############]
# Test:                   [----]
#          t1 t2 t3 t4 t5 t6 t7 t8

# Fold 4:
# Train:  [#################]
# Test:                      [----]
#          t1 t2 t3 t4 t5 t6 t7 t8

# Legend:
# [####] = Training data (expands each fold)
# [--]   = Test data (typically fixed size)

### Rolling Window

In [4]:
# Time Series Data: [==============================]
#                    t1  t2  t3  t4  t5  t6  t7  t8


# Fold 1:
# Train:   [####]
# Test:         [----]
#          t1 t2 t3 t4

# Fold 2:
# Train:     [#####]
# Test:            [----]
#          t1 t2 t3 t4 t5 t6

# Fold 3:
# Train:            [####]
# Test:                  [----]
#          t1 t2 t3 t4 t5 t6 t7 t8

# Fold 4:
# Train:               [####]
# Test:                     [----]
#          t1 t2 t3 t4 t5 t6 t7 t8

# Legend:
# [####] = Training data (expands each fold)
# [--]   = Test data (typically fixed size)

### Load Data

In [5]:
url = 'https://drive.google.com/uc?export=download&id=1qnX9GpiL5Ii1FEnHTIAzWnxNejWnilKp'
btcusdt = pd.read_csv(url, parse_dates=["open_time"], index_col='open_time')

btcusdt

,open,high,low,close,volume
open_time,,,,,
2020-11-27 00:00:00,17155.37,17418.93,17024.20,17400.00,15427.474
2020-11-27 01:00:00,17401.51,17465.00,17271.30,17309.94,16632.689
2020-11-27 02:00:00,17309.93,17328.09,17072.80,17102.38,16168.837
2020-11-27 03:00:00,17102.10,17277.86,17029.32,17084.05,13670.593
2020-11-27 04:00:00,17084.05,17194.00,17061.00,17079.56,10866.299
...,...,...,...,...,...
2025-11-10 19:00:00,105435.90,106000.00,105354.00,105767.20,2971.178
2025-11-10 20:00:00,105767.10,106249.60,105750.30,105956.70,3483.547
2025-11-10 21:00:00,105956.70,105973.90,105202.70,105583.50,3305.325


In [6]:
btcusdt['close_log_return'] = np.log(btcusdt['close']/btcusdt['close'].shift())
btcusdt

,open,high,low,close,volume,close_log_return
open_time,,,,,,
2020-11-27 00:00:00,17155.37,17418.93,17024.20,17400.00,15427.474,NaN
2020-11-27 01:00:00,17401.51,17465.00,17271.30,17309.94,16632.689,-0.005189
2020-11-27 02:00:00,17309.93,17328.09,17072.80,17102.38,16168.837,-0.012063
2020-11-27 03:00:00,17102.10,17277.86,17029.32,17084.05,13670.593,-0.001072
2020-11-27 04:00:00,17084.05,17194.00,17061.00,17079.56,10866.299,-0.000263
...,...,...,...,...,...,...
2025-11-10 19:00:00,105435.90,106000.00,105354.00,105767.20,2971.178,0.003138
2025-11-10 20:00:00,105767.10,106249.60,105750.30,105956.70,3483.547,0.001790
2025-11-10 21:00:00,105956.70,105973.90,105202.70,105583.50,3305.325,-0.003528


In [7]:
btcusdt['close_log_return_lag_1'] = btcusdt['close_log_return'].shift()
btcusdt['close_log_return_lag_2'] = btcusdt['close_log_return'].shift(2)
btcusdt['close_log_return_lag_3'] = btcusdt['close_log_return'].shift(3)
btcusdt = btcusdt.dropna()
btcusdt[['close_log_return','close_log_return_lag_1','close_log_return_lag_2','close_log_return_lag_3']]

,close_log_return,close_log_return_lag_1,close_log_return_lag_2,close_log_return_lag_3
open_time,,,,
2020-11-27 04:00:00,-0.000263,-0.001072,-0.012063,-0.005189
2020-11-27 05:00:00,0.010946,-0.000263,-0.001072,-0.012063
2020-11-27 06:00:00,0.001109,0.010946,-0.000263,-0.001072
2020-11-27 07:00:00,-0.010552,0.001109,0.010946,-0.000263
2020-11-27 08:00:00,-0.014575,-0.010552,0.001109,0.010946
...,...,...,...,...
2025-11-10 19:00:00,0.003138,-0.004455,0.006797,0.001525
2025-11-10 20:00:00,0.001790,0.003138,-0.004455,0.006797
2025-11-10 21:00:00,-0.003528,0.001790,0.003138,-0.004455


### Create Training Loop

In [8]:
def train_model(model, criterion, optimizer, X_train, y_train, X_test, y_test, no_epochs, verbose = True):
  # ensure training results are reproducable
  SEED = 99

  # Ensure Python’s hash-based operations are deterministic
  os.environ["PYTHONHASHSEED"] = str(SEED)

  # Set seeds for Python's built-in RNG, NumPy, and PyTorch
  random.seed(SEED)
  np.random.seed(SEED)

  torch.manual_seed(SEED)
  torch.cuda.manual_seed(SEED)          # For single-GPU setups
  torch.cuda.manual_seed_all(SEED)      # For multi-GPU setups

  # full batch gradient descent (only efficient for data sets <= local memory)
  for epoch in range(no_epochs):

      # Clear previously stored gradients (they accumulate by default)
      optimizer.zero_grad()

      # Forward pass: compute predictions
      y_pred = model(X_train)

      # Compute loss between predictions and true values
      loss = criterion(y_pred, y_train)

      # Backpropagation: compute gradients of loss w.r.t. parameters
      loss.backward()

      # Update model parameters using the computed gradients
      optimizer.step()

      # Print loss every 500 epochs
      if verbose and epoch % 500 == 0:
          print("Epoch:", epoch, "Loss:", loss.item())


  # print the the model's trained parameters
  if verbose:
    print("Trained weights:", model.weight.data)
    print("Trained bias:", model.bias.data)


### Create Time Series Split

We need to do this to split data into in-sample and out-sample that preserves time order.

In [9]:
def timesplit(df, train_size = 0.75):
  i = int(len(df) * train_size)
  return df[:i].copy(), df[i:].copy()

btcusdt_train, btcusdt_test = timesplit(btcusdt, train_size = 0.7)

### Empirically verify it's split correctly

In [10]:
btcusdt_train

,open,high,low,close,volume,close_log_return,close_log_return_lag_1,close_log_return_lag_2,close_log_return_lag_3
open_time,,,,,,,,,
2020-11-27 04:00:00,17084.05,17194.00,17061.00,17079.56,10866.299,-0.000263,-0.001072,-0.012063,-0.005189
2020-11-27 05:00:00,17079.55,17350.00,17078.78,17267.54,13783.564,0.010946,-0.000263,-0.001072,-0.012063
2020-11-27 06:00:00,17267.54,17316.32,17177.00,17286.70,9598.037,0.001109,0.010946,-0.000263,-0.001072
2020-11-27 07:00:00,17286.70,17297.90,17040.00,17105.25,13115.712,-0.010552,0.001109,0.010946,-0.000263
2020-11-27 08:00:00,17106.79,17116.04,16714.92,16857.75,31574.365,-0.014575,-0.010552,0.001109,0.010946
...,...,...,...,...,...,...,...,...,...
2024-05-16 20:00:00,65152.30,65270.10,65018.70,65248.00,3804.218,0.001469,0.000934,0.001765,-0.002423
2024-05-16 21:00:00,65247.90,65480.00,65204.20,65429.10,4205.893,0.002772,0.001469,0.000934,0.001765
2024-05-16 22:00:00,65429.00,65457.00,65100.00,65389.20,4533.237,-0.000610,0.002772,0.001469,0.000934


In [11]:
btcusdt_test

,open,high,low,close,volume,close_log_return,close_log_return_lag_1,close_log_return_lag_2,close_log_return_lag_3
open_time,,,,,,,,,
2024-05-17 01:00:00,65477.9,65478.0,65061.2,65311.0,6160.987,-0.002554,0.003983,-0.002626,-0.000610
2024-05-17 02:00:00,65311.1,65430.0,65146.5,65351.5,4190.566,0.000620,-0.002554,0.003983,-0.002626
2024-05-17 03:00:00,65351.4,65850.0,65300.3,65545.2,11900.518,0.002960,0.000620,-0.002554,0.003983
2024-05-17 04:00:00,65545.1,65600.0,65327.7,65422.7,5906.313,-0.001871,0.002960,0.000620,-0.002554
2024-05-17 05:00:00,65422.8,65675.0,65400.0,65674.9,4946.005,0.003848,-0.001871,0.002960,0.000620
...,...,...,...,...,...,...,...,...,...
2025-11-10 19:00:00,105435.9,106000.0,105354.0,105767.2,2971.178,0.003138,-0.004455,0.006797,0.001525
2025-11-10 20:00:00,105767.1,106249.6,105750.3,105956.7,3483.547,0.001790,0.003138,-0.004455,0.006797
2025-11-10 21:00:00,105956.7,105973.9,105202.7,105583.5,3305.325,-0.003528,0.001790,0.003138,-0.004455


### Train ML Model

In [12]:
features = ['close_log_return_lag_1','close_log_return_lag_2','close_log_return_lag_3']
target = 'close_log_return'

# Number of input features (1 in this case)
no_features = len(features)

# Simple linear regression model: y = Wx + b
model = nn.Linear(no_features, 1)

# Huber loss
criterion = nn.HuberLoss()

# Stochastic Gradient Descent optimizer
optimizer = optim.SGD(model.parameters(), lr=0.01)

# no of training iterations
no_epochs = 5000

# Our model's training (in-sample) and testing (out-sample) input converted into tensors
X_train = torch.tensor(btcusdt_train[features].values, dtype=torch.float32)
X_test  = torch.tensor(btcusdt_test[features].values, dtype=torch.float32)

# Create model's output (target) as tensors and add a column dimension (N → N×1)
y_train = torch.tensor(btcusdt_train[target].values, dtype=torch.float32).unsqueeze(1)
y_test  = torch.tensor(btcusdt_test[target].values, dtype=torch.float32).unsqueeze(1)

train_model(model, criterion, optimizer, X_train, y_train, X_test, y_test, no_epochs)

Epoch: 0 Loss: 0.09656275808811188
Epoch: 500 Loss: 3.313506385893561e-05
Epoch: 1000 Loss: 2.896555270126555e-05
Epoch: 1500 Loss: 2.896317710110452e-05
Epoch: 2000 Loss: 2.8960985218873248e-05
Epoch: 2500 Loss: 2.895879151765257e-05
Epoch: 3000 Loss: 2.8956601454410702e-05
Epoch: 3500 Loss: 2.8954407753190026e-05
Epoch: 4000 Loss: 2.8952217689948156e-05
Epoch: 4500 Loss: 2.895002944569569e-05
Trained weights: tensor([[0.2893, 0.2787, 0.0723]])
Trained bias: tensor([1.6122e-05])


### Retrive ML model's prediction for out-sample (test)

In [13]:
def test_model_predictions(model, X_test):
  model.eval()  # evaluation mode to make it more efficient
  with torch.no_grad():
      y_hat = model(X_test)
  # squeeze(1) changes tensor shape from (n, 1) to (n,)
  # Reduce 2-dimensional tensor to 1-dimensional tensor
  return y_hat.squeeze(1)

test_model_predictions(model, X_test)

tensor([ 0.0004,  0.0002, -0.0002,  ...,  0.0011, -0.0003,  0.0003])

### Add the prediction back to the test dataframe

In [ ]:
#y_hat - predicted values of what we want to predict vs actual

In [14]:
btcusdt_test['y_hat'] = test_model_predictions(model, X_test).numpy()
btcusdt_test

,open,high,low,close,volume,close_log_return,close_log_return_lag_1,close_log_return_lag_2,close_log_return_lag_3,y_hat
open_time,,,,,,,,,,
2024-05-17 01:00:00,65477.9,65478.0,65061.2,65311.0,6160.987,-0.002554,0.003983,-0.002626,-0.000610,0.000392
2024-05-17 02:00:00,65311.1,65430.0,65146.5,65351.5,4190.566,0.000620,-0.002554,0.003983,-0.002626,0.000198
2024-05-17 03:00:00,65351.4,65850.0,65300.3,65545.2,11900.518,0.002960,0.000620,-0.002554,0.003983,-0.000228
2024-05-17 04:00:00,65545.1,65600.0,65327.7,65422.7,5906.313,-0.001871,0.002960,0.000620,-0.002554,0.000861
2024-05-17 05:00:00,65422.8,65675.0,65400.0,65674.9,4946.005,0.003848,-0.001871,0.002960,0.000620,0.000345
...,...,...,...,...,...,...,...,...,...,...
2025-11-10 19:00:00,105435.9,106000.0,105354.0,105767.2,2971.178,0.003138,-0.004455,0.006797,0.001525,0.000732
2025-11-10 20:00:00,105767.1,106249.6,105750.3,105956.7,3483.547,0.001790,0.003138,-0.004455,0.006797,0.000173
2025-11-10 21:00:00,105956.7,105973.9,105202.7,105583.5,3305.325,-0.003528,0.001790,0.003138,-0.004455,0.001087


### Evaluate profitability of the model

ML metrics such as MSE and MAE doesn't tell us if the model is profitable or not. We are interested in calculated the expected value of each trade's gross log return.

In [ ]:
#LLN - where the EV kicks in

In [15]:
def eval_profitability(model, df_test, X_test):
  y_hat = test_model_predictions(model, X_test).numpy()
  df_test['y_hat'] = y_hat
  df_test['dir_signal'] = np.sign(y_hat)
  df_test['trade_log_return'] = df_test['dir_signal'] * df_test['close_log_return']
  df_test['cum_trade_log_return'] = df_test['trade_log_return'].cumsum()
  df_test['is_won'] = df_test['trade_log_return'] > 0

  return df_test['trade_log_return'].mean()

eval_profitability(model, btcusdt_test, X_test)

np.float64(-5.427826503906378e-05)

### Create function to evaluate model's profitability

It trains a model and then tests the model w.r.t profitability quantified as the expected value of the trade log return.

In [16]:
def eval_model_profitability(df_train, df_test, features, target):
  # Number of input features (1 in this case)
  no_features = len(features)

  # Simple linear regression model: y = Wx + b
  model = nn.Linear(no_features, 1)

  # Huber loss (robust to outliers compared to MSE)
  criterion = nn.HuberLoss()

  # Stochastic Gradient Descent optimizer
  optimizer = optim.SGD(model.parameters(), lr=0.01)

  X_train, X_test = df_train[features], df_test[features]
  y_train, y_test = df_train[target], df_test[target]

  # Convert train/test splits into PyTorch tensors
  X_train = torch.tensor(btcusdt_train[features].values, dtype=torch.float32)
  X_test  = torch.tensor(btcusdt_test[features].values, dtype=torch.float32)

  # Create target tensors and add a column dimension (N → N×1)
  y_train = torch.tensor(btcusdt_train[target].values, dtype=torch.float32).unsqueeze(1)
  y_test  = torch.tensor(btcusdt_test[target].values, dtype=torch.float32).unsqueeze(1)

  train_model(model, criterion, optimizer, X_train, y_train, X_test, y_test, no_epochs = 5000, verbose = False)

  return eval_profitability(model, df_test, X_test)

### Evaluate Model's Profitability for different Train/Test Split

In [17]:
  # implicitly defining an AR(3) model
  features = ['close_log_return_lag_1','close_log_return_lag_2','close_log_return_lag_3']
  target = 'close_log_return'

In [18]:
btcusdt_train, btcusdt_test = timesplit(btcusdt, train_size = 0.4)

eval_model_profitability(btcusdt_train, btcusdt_test, features, target)

np.float64(4.0235183905987034e-05)

In [19]:
btcusdt_train, btcusdt_test = timesplit(btcusdt, train_size = 0.5)

eval_model_profitability(btcusdt_train, btcusdt_test, features, target)

np.float64(3.0564637431533325e-05)

In [20]:
btcusdt_train, btcusdt_test = timesplit(btcusdt, train_size = 0.6)

eval_model_profitability(btcusdt_train, btcusdt_test, features, target)

np.float64(1.4089078228887257e-05)

In [21]:
btcusdt_train, btcusdt_test = timesplit(btcusdt, train_size = 0.7)

eval_model_profitability(btcusdt_train, btcusdt_test, features, target)

np.float64(4.1754408443673855e-05)

In [22]:
btcusdt_train, btcusdt_test = timesplit(btcusdt, train_size = 0.8)

eval_model_profitability(btcusdt_train, btcusdt_test, features, target)

np.float64(-1.0188936560753338e-05)

In [23]:
btcusdt_train, btcusdt_test = timesplit(btcusdt, train_size = 0.9)
eval_model_profitability(btcusdt_train, btcusdt_test, features, target)

np.float64(-7.539969202036555e-05)

### Cross Validation: Time Series Splits

In [ ]:
#Looping over train_split sizes
#Splitting the data by time
#Evaluate the model profitability the EV
#Add the train size
#take the mean of the EV
#how robust is this to different trainig sizes

In [24]:
train_sizes = []
evs = []
for train_split in [0.4 + 0.1 * i for i in range(6)]:
  btcusdt_train, btcusdt_test = timesplit(btcusdt, train_size = train_split)
  ev = eval_model_profitability(btcusdt_train, btcusdt_test, features, target)
  train_sizes.append(train_split)
  evs.append(ev)

cv_results = pd.DataFrame({'train_size': train_sizes, 'ev': evs})

In [25]:
cv_results

,train_size,ev
0,0.4,0.000040
1,0.5,0.000031
2,0.6,0.000014
3,0.7,0.000042
4,0.8,-0.000010
5,0.9,-0.000075


In [26]:
cv_results['ev'].mean()

np.float64(6.842446571493766e-06)

### Evaluate Rolling Window Cross Validation

In [27]:
no_rows = len(btcusdt_test)
no_rows

4344

In [28]:
hours_in_month = 24 * 30
hours_in_month

720

In [29]:
no_rows / hours_in_month

6.033333333333333

In [ ]:
#get the indices of the df to know what it's doing

In [30]:
for i in range(6):
  train_start, train_end = 724 * i, 724 * (i + 1)
  test_start, test_end = 724 * (i + 1), 724 * (i + 2)
  print(train_start, train_end, test_start, test_end)

0 724 724 1448
724 1448 1448 2172
1448 2172 2172 2896
2172 2896 2896 3620
2896 3620 3620 4344
3620 4344 4344 5068


In [31]:
btcusdt_train, btcusdt_test = btcusdt[:724].copy(), btcusdt[724:1448].copy()

In [32]:
btcusdt_train

,open,high,low,close,volume,close_log_return,close_log_return_lag_1,close_log_return_lag_2,close_log_return_lag_3
open_time,,,,,,,,,
2020-11-27 04:00:00,17084.05,17194.00,17061.00,17079.56,10866.299,-0.000263,-0.001072,-0.012063,-0.005189
2020-11-27 05:00:00,17079.55,17350.00,17078.78,17267.54,13783.564,0.010946,-0.000263,-0.001072,-0.012063
2020-11-27 06:00:00,17267.54,17316.32,17177.00,17286.70,9598.037,0.001109,0.010946,-0.000263,-0.001072
2020-11-27 07:00:00,17286.70,17297.90,17040.00,17105.25,13115.712,-0.010552,0.001109,0.010946,-0.000263
2020-11-27 08:00:00,17106.79,17116.04,16714.92,16857.75,31574.365,-0.014575,-0.010552,0.001109,0.010946
...,...,...,...,...,...,...,...,...,...
2020-12-27 03:00:00,26583.94,26663.00,26454.60,26590.96,9663.997,0.000264,-0.002028,-0.005128,0.009986
2020-12-27 04:00:00,26590.96,26843.00,26590.96,26760.01,9810.078,0.006337,0.000264,-0.002028,-0.005128
2020-12-27 05:00:00,26760.00,26967.06,26719.48,26905.88,10320.555,0.005436,0.006337,0.000264,-0.002028


In [33]:
btcusdt_test

,open,high,low,close,volume,close_log_return,close_log_return_lag_1,close_log_return_lag_2,close_log_return_lag_3
open_time,,,,,,,,,
2020-12-27 08:00:00,27655.40,27958.90,27342.44,27500.00,29192.883,-0.005630,0.000111,0.027361,0.005436
2020-12-27 09:00:00,27500.00,27882.79,27212.16,27830.75,25172.597,0.011956,-0.005630,0.000111,0.027361
2020-12-27 10:00:00,27830.74,27919.99,27705.00,27839.18,11007.336,0.000303,0.011956,-0.005630,0.000111
2020-12-27 11:00:00,27839.19,28459.84,27034.23,27679.15,66150.250,-0.005765,0.000303,0.011956,-0.005630
2020-12-27 12:00:00,27679.48,27679.61,26592.00,27234.02,50231.517,-0.016212,-0.005765,0.000303,0.011956
...,...,...,...,...,...,...,...,...,...
2021-01-26 07:00:00,31595.40,31768.07,31112.58,31640.70,18236.481,0.001433,-0.014343,0.001898,0.014813
2021-01-26 08:00:00,31640.71,31980.49,31518.96,31896.14,11939.225,0.008041,0.001433,-0.014343,0.001898
2021-01-26 09:00:00,31896.13,32094.88,31370.02,31813.46,15402.301,-0.002596,0.008041,0.001433,-0.014343


In [34]:
#It's independent of the algo to split the data
eval_model_profitability(btcusdt_train, btcusdt_test, features, target)

np.float64(0.0003481060312287868)

In [35]:
def eval_rolling_window_cv(df, features, target, window_size, no_iterations):
  window_no = []
  ev = []
  for i in range(no_iterations):
    train_start, train_end = window_size * i, window_size * (i + 1)
    test_start, test_end = window_size * (i + 1), window_size * (i + 2)
    df_train = df[train_start:train_end].copy()
    df_test = df[test_start:test_end].copy()

    window_no.append(i)
    ev.append(eval_model_profitability(df_train, df_test, features, target))

  return pd.DataFrame({'window_no': window_no, 'ev': ev})

In [36]:
rw_results = eval_rolling_window_cv(btcusdt, features, target, 724, 6)

In [37]:
rw_results

,window_no,ev
0,0,0.000348
1,1,0.000330
2,2,-0.000426
3,3,-0.000029
4,4,0.000720
5,5,-0.000555


In [ ]:
#A small positve edge using rolling window method

In [38]:
rw_results['ev'].mean()

np.float64(6.483478133362699e-05)

### Eval Expanding Window Cross Validation

In [39]:
window_size = 724
for i in range(6):
  train_start = 0
  train_end = window_size + i * window_size

  test_start = train_end
  test_end = test_start + window_size
  print((train_start, train_end, test_start, test_end))

(0, 724, 724, 1448)
(0, 1448, 1448, 2172)
(0, 2172, 2172, 2896)
(0, 2896, 2896, 3620)
(0, 3620, 3620, 4344)
(0, 4344, 4344, 5068)


In [40]:
def eval_expanding_window_cv(df, features, target, window_size, no_iterations):
  iteration_no =  []
  ev = []
  for i in range(no_iterations):
    train_start = 0
    train_end = window_size + i * window_size

    test_start = train_end
    test_end = test_start + window_size
    df_train = df[train_start:train_end].copy()
    df_test = df[test_start:test_end].copy()

    iteration_no.append(i+1)
    ev.append(eval_model_profitability(df_train, df_test, features, target))

  return pd.DataFrame({'iteration_no': iteration_no, 'ev': ev})

In [41]:
cv_result = eval_expanding_window_cv(btcusdt, features, target, window_size, 6)

In [42]:
cv_result

,iteration_no,ev
0,1,0.000348
1,2,0.000330
2,3,-0.000426
3,4,-0.000029
4,5,0.000720
5,6,-0.000555


In [ ]:
#What's the avg stat edge we have using CV?

In [43]:
cv_result['ev'].mean()

np.float64(6.483478133362699e-05)

### Conclusion

* Introduced Cross Validation
* Shown how to get more reliable and robust estimate of how well a ML model will perform on new, unseen data
* Showed examples of Cross Validation:
    * Time Series Split
    * Rolling Window
    * Expanding Window

In [ ]:
#If you use RW - then after a week retrain the weights

### Exercises

### Exercise 1.

In [46]:
#Training window size could be larger the test size
#Training window size is independent

In [44]:
# Try changing rolling window size to see if its results improves. Change both the train window size and train window size independently too.

### Exercise 2

In [ ]:
#Try different window sizes

In [45]:
# Try changing expanding window size to see if its results improves. Change both the train window size and train window size independently too.